In [2]:
import os, cv2, zipfile, requests, shutil, random

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

from pathlib import Path
from collections import Counter

# download and extract the dataset
DATA_DIR = Path("../data/raw")
DATA_DIR.mkdir(parents=True, exist_ok=True)

url = "https://www.kaggle.com/api/v1/datasets/download/nandanp6/cataract-image-dataset"
local_zip = DATA_DIR / "archive.zip"

if not local_zip.exists():
    print("Downloading dataset...")
    r = requests.get(url, stream=True)
    with open(local_zip, 'wb') as f:
        shutil.copyfileobj(r.raw, f)

    with zipfile.ZipFile(local_zip, 'r') as zip_ref:
        zip_ref.extractall(DATA_DIR)

In [ ]:
import glob
import pandas as pd
from PIL import Image
from tqdm import tqdm

# Define the dataset path
dataset_path = DATA_DIR / "processed_images/"

# Initialize a list to store image paths and labels
data = []

# Iterate through 'train' and 'test' directories
for split in ['train', 'test']:
    for label in ['normal', 'cataract']:
        folder_path = os.path.join(dataset_path, split, label)
        image_paths = glob.glob(os.path.join(folder_path, '*.png'))
        for img_path in image_paths:
            data.append({'path': img_path, 'label': 0 if label == 'normal' else 1, 'split': split})

# Create a DataFrame
df = pd.DataFrame(data)

# Shuffle the DataFrame
df = df.sample(frac=1).reset_index(drop=True)

# Display the first few rows
print(df.head())

..\data\raw\processed_images
                                                path  label  split
0  ..\data\raw\processed_images\test\normal\image...      0   test
1  ..\data\raw\processed_images\train\cataract\im...      1  train
2  ..\data\raw\processed_images\train\cataract\im...      1  train
3  ..\data\raw\processed_images\train\normal\imag...      0  train
4  ..\data\raw\processed_images\train\normal\imag...      0  train
